<font color='blue'> 
### <u>КОММЕНТАРИЙ РЕВЬЮЕРА</u>
</font>

<font color='blue'>
<br />
Илья, рад приветствовать тебя! <br />
<br />    

<br />
Меня зовут Николай. <br />И на этом этапе твоего движения к новой профессии от меня будут комментарии по написанному коду. <br />Чтобы меньше "мусорить эфир" и чтобы максимально наглядно отделяться от основного кода, есть предложение условиться в некоторых обозначениях:<br /> <br />
<u><b>ТАКОЙ ШРИФТ</b></u> - всегда начало комментария <br />
<font color='green'>такой шрифт</font> - комментарии о том, что всё ОК <br/>
<font color='orange'>такой шрифт</font> - комментарии о том, что всё ОК по результату,<br> однако есть на что обратить внимание в плане применения техник, или есть способы сделать более короткий или быстрый код <br/>
<font color='red'>такой шрифт</font> - комментарии о том, что есть критичный момент, влияющий на бизнес-результат проекта.<br/>
</font><br /><br />

# Доказываем что использование обратимой матрицы позволяет кодировать признаки влияния на точность прогнозирования модели линейной регрессии

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

## 1. Загрузка данных

In [2]:
data=pd.read_csv('./datasets/insurance.csv')

In [3]:
data.head(5)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


- Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.
- Целевой признак: количество страховых выплат клиенту за последние 5 лет.

In [4]:
data['Страховые выплаты'].value_counts()

0    4436
1     423
2     115
3      18
4       7
5       1
Name: Страховые выплаты, dtype: int64

In [5]:
data['Члены семьи'].value_counts()

1    1814
0    1513
2    1071
3     439
4     124
5      32
6       7
Name: Члены семьи, dtype: int64

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


## 2. Умножение матриц

In [7]:
#Поделим матрицу
train, test=train_test_split(data, shuffle=False, random_state=0)

In [8]:
class Linear_Regression:
    def fit(self, X_train, y_train):
        X = np.concatenate((np.ones((X_train.shape[0], 1)), X_train), axis=1)
        y = y_train
        w = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
        self.w = w[1:]
        self.w0 = w[0]
    def predict(self, X_test):
        return X_test@self.w + self.w0
    def score(self, X, y, set_name):
        print('r_2 на {} выборке: {:.3f}'.format(set_name,r2_score(y, self.predict(X))))

In [9]:
#Модель линейной регрессии
model_before=Linear_Regression()

#выделим призники и цели
X_train=train.drop('Страховые выплаты', axis=1)
y_train=train['Страховые выплаты']
X_test=test.drop('Страховые выплаты', axis=1)
y_test=test['Страховые выплаты']

model_before.fit(X_train,y_train)
model_before.score(X_train, y_train, 'обучающей')
model_before.score(X_test, y_test, 'тестовой')

r_2 на обучающей выборке: 0.426
r_2 на тестовой выборке: 0.423


Умножение признаков на обратимую матрицу

$X$ — матрица признаков (нулевой столбец состоит из единиц)

$P$ — обратимая матрица, на которую умножаются признаки

$$
XP
$$

In [10]:
#Создаём рандомную матрицу
random_matrix=np.array([])
while True:
    #Создаём рандомную матрицу
    random_matrix=np.random.random((X_train.shape[1],X_train.shape[1]))
    #Проверяем наличие обратной матрицы:
    try:
        #Если матрица обратима, то выходим из цикла
        np.linalg.inv(random_matrix)
        break
    except:
        continue

<font color='blue'> 
### <u>КОММЕНТАРИЙ РЕВЬЮЕРА</u>
</font>
<font color='green'>
ОК, надо проверять, чтобы матрица была обратимой. Ещё используют получение единичной матрицы)

In [11]:
#Модель линейной регрессии с изменёнными признаки
model_after=Linear_Regression()

X_train_changed=X_train@random_matrix
X_test_changed=X_test@random_matrix

model_after=Linear_Regression()
model_after.fit(X_train_changed,y_train)

model_after.score(X_train_changed, y_train, 'обучающей')
model_after.score(X_test_changed, y_test, 'тестовой')

r_2 на обучающей выборке: 0.426
r_2 на тестовой выборке: 0.423


<font color='blue'> 
### <u>КОММЕНТАРИЙ РЕВЬЮЕРА</u>
</font>
<font color='green'>
хорошая теория) В этом пункте нужно было только теорию. А вычисления все в пункте 4.

<div style="border:solid  green  2px; padding: 20px">
    <H2>Добавил</H2>
Вывод: качество линейной регресии не изменилось, так как параметры линейной регресии изменились в соответствии с соответствии с нашей обратимой матрицей коэффициентов. 
</div>

## 3. Алгоритм преобразования

пол - просто инвертируем 

возраст - вычитаем из 100

Зарплата - делим на 99

члены семьи смещаем на -3

In [13]:
#Целевая матрица
transform_data=data.iloc[:4,:-1].copy()
#Преобразуем пол и зарплату
transform_data=transform_data*[-1,1,1/99,1]

#Преобразуем всё остальное
transform_data=transform_data+[0,-100,0,-3]

transform_data=transform_data.values

In [14]:
#смотрим чтобы у нашей матрицы была обратная для дешиврования
np.linalg.inv(transform_data)

array([[-1.00000000e+00,  1.42708207e+00, -2.97689970e-01,
         3.89057751e-02],
       [-0.00000000e+00,  3.16413374e-02, -1.37993921e-02,
        -2.18844985e-02],
       [-0.00000000e+00,  4.99513678e-03, -3.12948328e-03,
        -6.01823708e-04],
       [-0.00000000e+00, -3.95653495e-01, -2.28024316e-01,
         4.75379939e-01]])

<font color='blue'> 
### <u>КОММЕНТАРИЙ РЕВЬЮЕРА</u>
</font>
<font color='green'>
Не понял этот пункт((( ведь матрицу рандомную уже создали и всё. Зачем не каждый отдельно преобразовывать?<br>


тут как раз и нужен план действий дл пункта 4) Делай раз, делай два, делай три..<br>
1) создаём рандомную матрицу<br>
2) проверяем её на обратимость (создаём единичную матрицу)<br>
3) Сщздаём новую матрицу данных<br>
4) Ищем метрики качества на исходных данных и на преобразованных<br>
5) делаем вывод<br>
    
Ну и обоснование этому на основе п.2

<div style="border:solid  green  2px; padding: 20px">
    <H2>Ответ</H2>
    В задании было написано Предложите алгоритм преобразования данных для решения задачи. Я и предложил.
    
    Вывод: качество модели не меняется после исользования обратимой матрицы. Качество не меняется потому, что матрицы отличаются друг от друга на величину обратной матрицы коэффициентов. Обратная матрица нужна чтобы вернуть масштаб.
    
    Проблема в том что мне не очень понравилась идея использования рандомных обратимых матриц для шифрования данных. Николай, в слаке мне не ответили на мой вопрос, может ты мне ответишь. Использование рандомной матрицы по сути смешивает между собой строки и столбцы исходной матрицы. Соответственно, на выходе мы получаем малоожидаемый результат, который нельзя расшифровать без использования обратной матрицы коэффициентов. Можно ли как-то элегантно зашифровать исходную матрицу в соответствии с теми пунктами, что я привёл в начале 3 раздела, чтобы не было необходимости сохранения исходной матрицы коэффициентов (генерировать матрицы, в которых определить не равен нулю, эт оне элегантно) и строки со столбцами не перемешивались?
</div>

## 4. Проверка алгоритма

Упрощение:

$$
(AB)^T=B^TA^T
$$

$$
((AB)^T)^{-1}=((AB)^{-1})^{T}
$$

$$
(AB)^{-1}=B^{-1}A^{-1}
$$

$$
A(A)^{-1}=(A)^{-1}A=E
$$

$E$ — Единичная матрица

$$
EA=AE=A
$$

$$
\frac{d}{dX}X^Ta=a
$$

$$
\frac{d}{dX}X^TAX=(A+A^T)X
$$

Зависимсоть целевого показателя $y_i$ от регрессоров i-го наблюдения может быть выражена через уравнение линейной регрессии вида:

$$
f(w,x_i)=w_0+w_1x_{1,i}+...+w_kx_{k,i}
$$

где $x_i$ - i-ое значение регрессора от 1 до n

k - количество регрессоров

w - угловые коэффициенты, которые представляют величину, на которую изменится расчётный целевой показатель в среднем при изменении регрессора

Оценка качества апроксимирующей функции осуществляеится методом наименьших квадратов:

$$
Err=\sum\limits_{i=1}^n{(y_i-f(x_i))^2} \rightarrow min
$$

Запишем уравнение в матричной форме:

$$
Err={(\overrightarrow{y}-X\overrightarrow{w})^2} \rightarrow min
$$

Аналогичным образом записывается уравнение, дополненное весовой матрицей
$$
Err={(\overrightarrow{y}-XA\overrightarrow{w})^2} \rightarrow min
$$

Размеры векторов и матриц:
$X:(n,k)$, $\overrightarrow{w}:(k,1)$, $\overrightarrow{y}:(n,1)$, $A:(k,k)$

При этом учитываем, что первый коэффициет $w_0$ в уравнении линейной регрессии не умножается ни на один регрессор. Чтобы можно было считать всё матрицами, каждое i-ое значение ргегрессора $w_0$ приравняем к единице.

Начнём раскрывать скобки на формулы с дополнительной матрицей:

$$
(\overrightarrow{y}-XA\overrightarrow{w})^2=
(\overrightarrow{y}-XA\overrightarrow{w})^T(\overrightarrow{y}-XA\overrightarrow{w})=
(\overrightarrow{y})^T\overrightarrow{y}-(\overrightarrow{y})^TXA\overrightarrow{w}-(XA\overrightarrow{w})^T\overrightarrow{y}+(XA\overrightarrow{w})^TXA\overrightarrow{w}
$$

Транспонирование обусловлено тем, что в скобках получается вектор

Подготовим уравнение к дифференцированию:

$$
(\overrightarrow{y})^TXA\overrightarrow{w}=((X\overrightarrow{w})^T\overrightarrow{y})^T=(XA\overrightarrow{w})^T\overrightarrow{y}=\overrightarrow{w}^TA^TX^T\overrightarrow{y}=const
$$

$$
(XA\overrightarrow{w})^T\overrightarrow{y}=\overrightarrow{y}^TXA\overrightarrow{w}=const
$$

$$
(XA\overrightarrow{w})^TXA\overrightarrow{w}=\overrightarrow{w}^TA^TX^TXA\overrightarrow{w}
$$

так как результат преобразований равен const, Т можно опустить и получаем:

$$
Err=(\overrightarrow{y})^T\overrightarrow{y}-2\overrightarrow{w}^TA^TX^T\overrightarrow{y}+\overrightarrow{w}^TA^TX^TXA\overrightarrow{w}
$$

Решение задачи минимизации достигается бутём поиска минимума ошибки по частной производной от весов:
$$
Err(X,\overrightarrow{y},\overrightarrow{w})\rightarrow min_{\overrightarrow{w}}\leftrightarrow \frac{dErr(X,\overrightarrow{y},\overrightarrow{w})}{d\overrightarrow{w}}=0
$$

$$
\frac{dErr(X,\overrightarrow{y},\overrightarrow{w})}{d\overrightarrow{w}}=\frac{d}{d\overrightarrow{w}}(\overrightarrow{y}^T\overrightarrow{y}-2\overrightarrow{w}^TA^TX^T\overrightarrow{y}+\overrightarrow{w}^TA^TX^TXA\overrightarrow{w})=\frac{\overrightarrow{y}^T\overrightarrow{y}}{d\overrightarrow{w}} - 
\frac{2\overrightarrow{w}^TA^TX^T\overrightarrow{y}}{d\overrightarrow{w}}+
\frac{\overrightarrow{w}^TA^TX^TXA\overrightarrow{w}}{d\overrightarrow{w}}
$$

Распишем каждую производную по отдельности:

$$
\frac{\overrightarrow{y}^T\overrightarrow{y}}{d\overrightarrow{w}}=0
$$

$$
\frac{2\overrightarrow{w}^TA^TX^T\overrightarrow{y}}{d\overrightarrow{w}}=2A^TX^T\overrightarrow{y}
$$

$$
\frac{\overrightarrow{w}^TA^TX^TXA\overrightarrow{w}}{d\overrightarrow{w}}=(A^TX^TXA+(A^TX^TXA)^T)\overrightarrow{w}=2A^TX^TXA\overrightarrow{w}
$$

Продолжим:
$$
\frac{dErr(X,\overrightarrow{y},\overrightarrow{w})}{d\overrightarrow{w}}=-2A^TX^T\overrightarrow{y}+2A^TX^TXA\overrightarrow{w}=0
$$


Качество линейной модели не изменилось, так как параметры линейной линейной регрессии до и после преобразований матрицы под обучение не поменялись.

Вектор весов линейной регрессии на этапе обучения вычисляется по следующей формуле:

$$
w = (X^T X)^{-1} X^T y
$$

Докажем что использование обратимой матрицы изменяет веса:

$$
((XP)^T (XP))^{-1} (XP)^T y = (X^T X)^{-1} X^T y
$$



Преобразуем левую часть:

$$
(XP)^{-1}((XP)^T)^{-1} (XP)^T y = (X^T X)^{-1} X^T y
$$

$$
(XP)^{-1}E y = (X^T X)^{-1} X^T y
$$

$$
(XP)^{-1} y = (X^T X)^{-1} X^T y
$$

Преобразуем правую часть:

$$
(XP)^{-1} y = (X)^{-1}(X^T)^{-1} X^T y
$$

$$
(XP)^{-1} y = (X)^{-1}E y
$$

$$
P^{-1}(X)^{-1} y = (X)^{-1} y
$$

Таким образом:

$$
\frac{w_{до}}{w_{после}}=P^{-1}
$$


In [15]:
#Модель линейной регрессии с изменёнными признаки
model_final=Linear_Regression()

X_final_train=X_train@transform_data
X_final_test=X_test@transform_data

model_final=Linear_Regression()
model_final.fit(X_final_train,y_train)

model_final.score(X_final_train, y_train, 'обучающей')
model_final.score(X_final_test, y_test, 'тестовой')

r_2 на обучающей выборке: 0.426
r_2 на тестовой выборке: 0.423


**Вывод**: точность модели не поменялась, так как весовые коэффициенты линейной регресиии изменились в соответствии с матрицой прпеобразования

<font color='blue'> 
### <u>КОММЕНТАРИЙ РЕВЬЮЕРА</u>
</font>
<font color='green'>
Илья, и с линейной алгеброй ты справился (почти)<br>
Всё, в общем-то, по делу, со смыслом. Доказательство в п.2 - классно)<br>
Да и весь проект - на уровне. Ответ и доказательство - верен.<br>
    
Надо только привести всё в порядок (не принципиально, но ты говорил, что готов исправить))):
<font color='red'>
1) вычисления из п2 перенести в п.4. <br>
2) п.3 - нужен только порядок действий и обснование ему.<br>
Скорректируй, пожалуйста)<br><font color='green'><font color='green'>
Спасибо) Буду ждать корректировок.

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Выполнен шаг 1: данные загружены
- [ ]  Выполнен шаг 2: получен ответ на вопрос об умножении матриц
    - [ ]  Указан правильный вариант ответа
    - [ ]  Вариант обоснован
- [ ]  Выполнен шаг 3: предложен алгоритм преобразования
    - [ ]  Алгоритм описан
    - [ ]  Алгоритм обоснован
- [ ]  Выполнен шаг 4: алгоритм проверен
    - [ ]  Алгоритм реализован
    - [ ]  Проведено сравнение качества моделей до и после преобразования